In [1]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=20,10

from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv("all_data.csv", encoding='utf-8')
# get data csv files as a pandas DataFrame

In [3]:
# preview the top-5 rows of the data
df.head()

,state,bachelor,graduate,hs,year,state_legislature,governor,lt_governor,labour_force_participation,mngmt_job_wage_gap,bachelor_wage_gap,ab_legality,rape_rate,life_exp_m_f,sex_ratio_birth,abortion_rate
0,Alabama,1.173,1.234,1.001,2020,0.157,1.0,0.0,0.782,0.640,25.1,0,32.7,0.918,93.6,113.548
1,Alaska,1.204,1.181,0.717,2020,0.367,0.0,0.0,0.795,0.779,18.1,1,154.8,0.938,109.2,164.547
2,Arizona,1.080,1.017,0.969,2020,0.389,0.0,0.0,0.789,0.701,23.2,1,44.0,0.923,98.8,184.862
3,Arkansas,1.187,1.224,0.921,2020,0.259,0.0,0.0,0.837,0.705,20.1,0,73.5,0.928,96.4,104.638
4,California,1.101,1.021,0.927,2020,0.308,0.0,1.0,0.764,0.734,18.6,1,34.2,0.929,98.9,16.075


In [5]:
round(df.describe() ,3)
#/Users/xuguangjie/Desktop/final-project-233-not-found-main/eda.ipynb

,bachelor,graduate,hs,year,state_legislature,governor,lt_governor,labour_force_participation,mngmt_job_wage_gap,bachelor_wage_gap,ab_legality,rape_rate,life_exp_m_f,sex_ratio_birth,abortion_rate
count,500.000,500.000,500.000,500.000,500.000,500.000,500.000,500.000,500.000,500.000,500.000,500.000,500.000,500.000,500.000
mean,1.135,1.105,0.947,2015.500,0.254,0.128,0.248,0.829,0.711,20.284,0.720,41.277,0.939,97.717,226.896
std,0.059,0.134,0.068,2.875,0.076,0.334,0.432,0.041,0.044,3.644,0.449,17.820,0.008,3.118,268.040
min,0.994,0.576,0.641,2011.000,0.094,0.000,0.000,0.732,0.568,10.200,0.000,11.400,0.912,93.100,2.714
25%,1.098,1.029,0.917,2013.000,0.199,0.000,0.000,0.803,0.683,18.200,0.000,31.075,0.934,95.400,121.059
50%,1.127,1.104,0.955,2015.500,0.253,0.000,0.000,0.826,0.709,20.100,1.000,38.600,0.940,97.100,188.740
75%,1.163,1.182,0.993,2018.000,0.304,0.000,0.000,0.862,0.736,22.225,1.000,47.100,0.944,99.100,257.928
max,1.394,1.521,1.113,2020.000,0.540,1.000,1.000,0.929,0.841,32.100,1.000,161.600,0.958,111.200,3700.745


In [39]:
import plotly.graph_objs as go
import pandas as pd

import_feat = ['state_legislature','mngmt_job_wage_gap','rape_rate','life_exp_m_f','abortion_rate']

# Calculate the rank percentage for each state in 2011
df['pct_rank'] = df[df['year'] == 2011]['state_legislature'].rank(pct=True)
# print(df['pct_rank'])

# Divide states into three groups based on rank percentage
df['group'] = pd.cut(df['pct_rank'], bins=[0, 0.1, 0.45, 0.55, 0.9, 1.0], labels=[r'前10%', r'10%-45%', r'45%-55%', r'55%-90%', r'后10%'])
# print(df[df['group'] == r'前10%'])
# Process each state's data individually and store it in a dictionary
states_data = {}
for state, data in df.groupby('state'):
    # Sort the data by year
    data = data.sort_values('year')
    # Construct Plotly's Trace object
    trace = go.Scatter(
        x=data['year'],
        y=data['state_legislature'],
        name=state,
        line=dict(width=2),
        mode='lines',
        text=data['state_legislature'],
        hovertemplate='<b>%{text:.2f}%</b><brstate: %{name}<br>year: %{x}'
    )
    # Assign the corresponding color according to which group the state belongs to
    if r'前10%' in data['group'].unique():
        trace.update(line=dict(color='#750D86'))
        states_data[state] = trace
    elif r'后10%' in data['group'].unique():
        trace.update(line=dict(color='#FEAF16'))
        states_data[state] = trace

    elif r'45%-55%' in data['group'].unique():
        trace.update(line=dict(color='#00AB8B'))
        states_data[state] = trace
    
    # Store a Trace object for each state

# Construct Plotly's Figure object
fig = go.Figure(list(states_data.values()))
fig.add_shape(
    type="line",
    x0=2011,
    y0=0.09,
    x1=2020,
    y1=0.09,
    line=dict(
        color="Red",
        width=2,
        dash="dashdot"
    )
)
fig.add_shape(
    type="line",
    x0=2011,
    y0=0.23,
    x1=2020,
    y1=0.23,
    line=dict(
        color="Red",
        width=2,
        dash="dashdot"
    )
)
# Add a chart title and axis labels
fig.update_layout(title='state_legislature',
                xaxis_title='year',
                yaxis_title='state_legislature')

# show chart
fig.show()


state                          Nevada
bachelor                        1.072
graduate                        1.013
hs                              0.928
year                             2020
state_legislature                0.54
governor                          0.0
lt_governor                       1.0
labour_force_participation      0.809
mngmt_job_wage_gap              0.763
bachelor_wage_gap                25.1
ab_legality                         1
rape_rate                        59.0
life_exp_m_f                    0.928
sex_ratio_birth                 100.7
abortion_rate                 277.085
pct_rank                          NaN
group                             NaN
Name: 27, dtype: object
state                          Nevada
bachelor                        1.072
graduate                        1.013
hs                              0.928
year                             2020
state_legislature                0.54
governor                          0.0
lt_governor               

<h4> State legislature</h4>

* Proportion of women in legislature.
* In general, the trend is steady before 2018, with a slight increase after.
    * Colorado: stays the top 1 in female proportion in state legislature. 2018-2019 shows a sharp increase.
    * Hawaii: moved from the top 10% to the middle range (45%-55%)
    * Alaska: moved from the last in the middle range to the top 10% range
    * Oklahoma: shows a sharp 

In [21]:
import_feat = ['state_legislature','mngmt_job_wage_gap','rape_rate','life_exp_m_f','abortion_rate']

# Calculate the rank percentage for each state in 2011
df['pct_rank'] = df[df['year'] == 2011]['mngmt_job_wage_gap'].rank(pct=True)

# Divide states into three groups based on rank percentage
df['group'] = pd.cut(df['pct_rank'], bins=[0, 0.1, 0.45, 0.55, 0.9, 1.0], labels=[r'前10%', r'10%-45%', r'45%-55%', r'55%-90%', r'后10%'])

# Process each state's data individually and store it in a dictionary
states_data = {}
for state, data in df.groupby('state'):
    # Sort the data by year
    data = data.sort_values('year')
    # Construct Plotly's Trace object
    trace = go.Scatter(
        x=data['year'],
        y=data['mngmt_job_wage_gap'],
        name=state,
        line=dict(width=2),
        mode='lines',
        text=data['mngmt_job_wage_gap'],
        hovertemplate='<b>%{text:.2f}%</b><brstate: %{name}<br>year: %{x}'
    )
    # Assign the corresponding color according to which group the state belongs to
    if r'前10%' in data['group'].unique():
        
        trace.update(line=dict(color='#750D86'))
        states_data[state] = trace

    elif r'后10%' in data['group'].unique():
        trace.update(line=dict(color='#FEAF16'))
        states_data[state] = trace

    elif r'45%-55%' in data['group'].unique():
        trace.update(line=dict(color='#00AB8B'))
        states_data[state] = trace

    # Store a Trace object for each state

# Construct Plotly's Figure object
fig = go.Figure(list(states_data.values()))

# Add a chart title and axis labels
fig.update_layout(title='mngmt_job_wage_gap',
                xaxis_title='year',
                yaxis_title='mngmt_job_wage_gap')

# show chart
fig.show()


In [24]:
import_feat = ['state_legislature','mngmt_job_wage_gap','rape_rate','life_exp_m_f','abortion_rate']

# Calculate the rank percentage for each state in 2011
df['pct_rank'] = df[df['year'] == 2011]['rape_rate'].rank(pct=True)

# Divide states into three groups based on rank percentage
df['group'] = pd.cut(df['pct_rank'], bins=[0, 0.1, 0.45, 0.55, 0.9, 1.0], labels=[r'前10%', r'10%-45%', r'45%-55%', r'55%-90%', r'后10%'])

# Process each state's data individually and store it in a dictionary
states_data = {}
for state, data in df.groupby('state'):
    # Sort the data by year
    data = data.sort_values('year')
    # Construct Plotly's Trace object
    trace = go.Scatter(
        x=data['year'],
        y=data['rape_rate'],
        name=state,
        line=dict(width=2),
        mode='lines',
        text=data['rape_rate'],
        hovertemplate='<b>%{text:.2f}%</b><brstate: %{name}<br>year: %{x}'
    )
    # Assign the corresponding color according to which group the state belongs to
    if r'前10%' in data['group'].unique():
        
        trace.update(line=dict(color='#750D86'))
        states_data[state] = trace

    elif r'后10%' in data['group'].unique():
        trace.update(line=dict(color='#FEAF16'))
        states_data[state] = trace

    elif r'45%-55%' in data['group'].unique():
        trace.update(line=dict(color='#00AB8B'))
        states_data[state] = trace

    # Store a Trace object for each state

# Construct Plotly's Figure object
fig = go.Figure(list(states_data.values()))

# Add a chart title and axis labels
fig.update_layout(title='rape_rate',
                xaxis_title='year',
                yaxis_title='rape_rate')

# show chart
fig.show()

In [25]:
import_feat = ['state_legislature','mngmt_job_wage_gap','rape_rate','life_exp_m_f','abortion_rate']

# Calculate the rank percentage for each state in 2011
df['pct_rank'] = df[df['year'] == 2011]['life_exp_m_f'].rank(pct=True)

# Divide states into three groups based on rank percentage
df['group'] = pd.cut(df['pct_rank'], bins=[0, 0.1, 0.45, 0.55, 0.9, 1.0], labels=[r'前10%', r'10%-45%', r'45%-55%', r'55%-90%', r'后10%'])

# Process each state's data individually and store it in a dictionary
states_data = {}
for state, data in df.groupby('state'):
    # Sort the data by year
    data = data.sort_values('year')
    # Construct Plotly's Trace object
    trace = go.Scatter(
        x=data['year'],
        y=data['life_exp_m_f'],
        name=state,
        line=dict(width=2),
        mode='lines',
        text=data['life_exp_m_f'],
        hovertemplate='<b>%{text:.2f}%</b><brstate: %{name}<br>year: %{x}'
    )
    # Assign the corresponding color according to which group the state belongs to
    if r'前10%' in data['group'].unique():
        
        trace.update(line=dict(color='#750D86'))
        states_data[state] = trace

    elif r'后10%' in data['group'].unique():
        trace.update(line=dict(color='#FEAF16'))
        states_data[state] = trace

    elif r'45%-55%' in data['group'].unique():
        trace.update(line=dict(color='#00AB8B'))
        states_data[state] = trace

    # Store a Trace object for each state

# Construct Plotly's Figure object
fig = go.Figure(list(states_data.values()))

# Add a chart title and axis labels
fig.update_layout(title='life_exp_m_f',
                xaxis_title='year',
                yaxis_title='life_exp_m_f')

# show chart
fig.show()

In [26]:
import_feat = ['state_legislature','mngmt_job_wage_gap','rape_rate','life_exp_m_f','abortion_rate']

# Calculate the rank percentage for each state in 2011
df['pct_rank'] = df[df['year'] == 2011]['abortion_rate'].rank(pct=True)

# Divide states into three groups based on rank percentage
df['group'] = pd.cut(df['pct_rank'], bins=[0, 0.1, 0.45, 0.55, 0.9, 1.0], labels=[r'前10%', r'10%-45%', r'45%-55%', r'55%-90%', r'后10%'])

# Process each state's data individually and store it in a dictionary
states_data = {}
for state, data in df.groupby('state'):
    # Sort the data by year
    data = data.sort_values('year')
    # Construct Plotly's Trace object
    trace = go.Scatter(
        x=data['year'],
        y=data['abortion_rate'],
        name=state,
        line=dict(width=2),
        mode='lines',
        text=data['abortion_rate'],
        hovertemplate='<b>%{text:.2f}%</b><brstate: %{name}<br>year: %{x}'
    )
    # Assign the corresponding color according to which group the state belongs to
    if r'前10%' in data['group'].unique():
        
        trace.update(line=dict(color='#750D86'))
        states_data[state] = trace

    elif r'后10%' in data['group'].unique():
        trace.update(line=dict(color='#FEAF16'))
        states_data[state] = trace

    elif r'45%-55%' in data['group'].unique():
        trace.update(line=dict(color='#00AB8B'))
        states_data[state] = trace

    # Store a Trace object for each state

# Construct Plotly's Figure object
fig = go.Figure(list(states_data.values()))

# Add a chart title and axis labels
fig.update_layout(title='abortion_rate',
                xaxis_title='year',
                yaxis_title='abortion_rate')

# show chart
fig.show()